In [1]:
import cv2
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg')
import torch
from torchvision import transforms
import os
os.getcwd()

'/home/matthias/workspace/Coding/ML_gh_repo_personal/MachineLearning_Python/yolo_v7'

In [2]:
#os.chdir('workspace/Coding/ML_gh_repo_personal/MachineLearning_Python/yolo_v7')

In [3]:
#os.getcwd()

In [4]:
from utils.datasets import letterbox
from models.experimental import attempt_load 
from utils.general import non_max_suppression_kpt
from utils.plots import output_to_keypoint
from utils.pose import show_image, plot_pose_prediction, make_pose_prediction, add_pose_in_video_file

In [5]:
# set gpu device if possible
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device:', device)

Device: cuda:0


In [6]:
# Load model
#model = attempt_load('MachineLearning_Python/yolo_v7/weights_pretrained_models/yolov7_w6_pretrained_pose.pt', map_location=device)
model = attempt_load('weights_pretrained_models/yolov7_w6_pretrained_pose.pt', map_location=device) 

Checking file:  weights_pretrained_models/yolov7_w6_pretrained_pose.pt
File exists?  True
Fusing layers... 


/home/matthias/anaconda3/envs/cv_pose/lib/python3.11/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /workspace/croot/pytorch_1692381528408/work/aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [7]:
# Switch to evaluation mode, map_location=device
model.eval()

print('Number of classes:', model.yaml['nc'])
print('Number of keypoints:', model.yaml['nkpt'])

Number of classes: 1
Number of keypoints: 17


In [8]:
# read original image
orig_img = cv2.imread('inference/images/image1.jpg')
orig_img = cv2.cvtColor(orig_img, cv2.COLOR_BGR2RGB)
print('Original image', orig_img.shape)

Original image (536, 640, 3)


In [12]:
#plt.figure(figsize=(6,6))
plt.imshow(orig_img)
plt.axis('off')
plt.savefig('fig.png')
#plt.show()

In [11]:
# resize and pad
img = letterbox(orig_img, 640, stride=64, auto=True)[0]
print('Resized image', img.shape)

#plt.figure(figsize=(6,6))
plt.imshow(img)
plt.axis('off')
plt.savefig('fig_resized.png')
#plt.show()

Resized image (576, 640, 3)


In [ ]:
print('Anchors:', model.yaml['anchors'])

In [ ]:
# transform to tensor
image_tensor = transforms.ToTensor()(img)
# add dimension
image_tensor = torch.unsqueeze(image_tensor, 0)
print('Transformed to tensor image:', image_tensor.shape)
# send the picture to the calculating device
image_tensor = image_tensor.to(device).float()

with torch.no_grad():
    pred, _ = model(image_tensor)
print('Predictions shape:', pred.shape)


In [ ]:
pred = non_max_suppression_kpt(pred, 
                               conf_thres=0.25, 
                               iou_thres=0.65, 
                               nc=model.yaml['nc'], 
                               nkpt=model.yaml['nkpt'], 
                               kpt_label=True)
print('Detected poses:', len(pred))
print('Prediction shape:', pred[0].shape)

In [ ]:
pred = output_to_keypoint(pred)
plot_pose_prediction(img, pred)

plt.figure(figsize=(6,6))
plt.imshow(img)
plt.axis('off')
plt.show()


In [ ]:
img = cv2.cvtColor(cv2.imread('breakdance.jpg'), cv2.COLOR_BGR2RGB)
pred = make_pose_prediction(model, img)
plot_pose_prediction(img, pred, show_bbox=False)

In [ ]:
plt.figure(figsize=(18,18))
plt.imshow(img)
plt.axis('off')
plt.show()

add_pose_in_video_file('board.mp4', 'board_out.mp4')